In [1]:
# !python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 7.8MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# !pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.2 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 809 kB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=222156 sha256=b1ac74ed1952d812d953038aaf031684fb1b6be7f2c2e5b4241848926ebb74d9
  Stored in directory: /home/jovyan/.cache/pip/wheels/88/63/63/f727a62aaba1e0fe13fe549e1b7538e9b8a2bc43dcae8138c8
  Created wheel for docstring-parser: filename=docstring_parser-0.7.3-py3-none-any.whl siz

### after installing the necessary packages, please restart kernel before continuing

In [1]:
import kfp
from kfp import dsl
import kfp.components as comp

### the working directory is /home/jovyan, ensure you create the folder to save your outputs before running, "store" was the folder created here

In [5]:
# create  directory for outputs.
output_dir = "/home/jovyan/KubeflowTraining/Day 2/Kubeflow components and pipeline/Labs/TensorFlow/store/"

### Note that the functions created are called here to ensure there are no errors before it is been truned to a component and compiled as a part of the pipeline


### Defining the function that gets data from the source

In [6]:
def obtain_data(data_path, working_data):
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install','pandas==0.23.4'])
    import pandas as pd
    
     #reading the data from its source
    data = pd.read_csv("https://raw.githubusercontent.com/MavenCode/KubeflowTraining/master/Data/Churn_Modelling.csv")
    #Save the data as a pickle file to be used by the preprocess component.
    with open(f'{data_path}/{working_data}', 'wb') as f:
        pickle.dump(data, f)

In [7]:
obtain_data(output_dir, "working_data")

### Defining the preprocess function

In [8]:
def preprocess(data_path,working_data,train_data,test_data):
    import pickle
    # import Library
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install','scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install','pandas==0.23.4'])
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler 

    #loading the working data
    with open(f'{data_path}/{working_data}', 'rb') as f:
        data = pickle.load(f)

    #dropping some columns that are not needed
    data = data.drop(columns=['RowNumber','CustomerId','Surname'], axis=1)
    #data features
    X = data.iloc[:,:-1]
    #target data
    y = data.iloc[:,-1:]   
    #encoding the categorical columns
    le = LabelEncoder()
    ohe = OneHotEncoder()
    X['Gender'] = le.fit_transform(X['Gender'])
    geo_df = pd.DataFrame(ohe.fit_transform(X[['Geography']]).toarray())

    #getting feature name after onehotencoding
    geo_df.columns = ohe.get_feature_names(['Geography'])

    #merging geo_df with the main data
    X = X.join(geo_df) 
    #dropping the old columns after encoding
    X.drop(columns=['Geography'], axis=1, inplace=True)

    #splitting the data 
    X_train,X_test,y_train,y_test = train_test_split( X,y, test_size=0.2, random_state = 42)
    #feature scaling
    sc =StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    #saving the values from the dataframe
    y_train = y_train.values
    y_test = y_test.values
    
    #Save the train_data as a pickle file to be used by the train component.
    with open(f'{data_path}/{train_data}', 'wb') as f:
        pickle.dump((X_train,  y_train), f)
        
    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/{test_data}', 'wb') as f:
        pickle.dump((X_test,  y_test), f)
    
    return(print('Done!'))

In [9]:
preprocess(output_dir,"working_data", "train_data","test_data")

Done!


### Defining the train function

In [10]:
def train_tensorflow(data_path,train_data, model):
    import pickle
    # import Library
    import numpy as np
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense

    #loading the train data
    with open(f'{data_path}/{train_data}', 'rb') as f:
        train_data = pickle.load(f)
    # Separate the X_train from y_train.
    X_train, y_train = train_data
    
    #initializing the classifier model with its input, hidden and output layers
    classifier = Sequential()
    classifier.add(Dense(units = 16, activation='relu', input_dim=12,))
    classifier.add(Dense(units = 8, activation='relu'))
    classifier.add(Dense(units = 1, activation='sigmoid'))
    #Compiling the classifier model with Stochastic Gradient Desecnt
    classifier.compile(optimizer = 'adam', loss='binary_crossentropy' , metrics =['accuracy'])
    #fitting the model
    classifier.fit(X_train, y_train, batch_size=10, epochs=150)
    #saving the model
    classifier.save(f'{data_path}/{model}')    

In [11]:
train_tensorflow(output_dir, "train_data","model")

Train on 8000 samples
Epoch 1/150
8000/8000 [==============================] - 7s 885us/sample - loss: 0.4606 - accuracy: 0.7950
Epoch 2/150
8000/8000 [==============================] - 5s 664us/sample - loss: 0.4105 - accuracy: 0.8171
Epoch 3/150
8000/8000 [==============================] - 5s 674us/sample - loss: 0.3883 - accuracy: 0.8270
Epoch 4/150
8000/8000 [==============================] - 5s 664us/sample - loss: 0.3742 - accuracy: 0.8371
Epoch 5/150
8000/8000 [==============================] - 5s 686us/sample - loss: 0.3648 - accuracy: 0.8482
Epoch 6/150
8000/8000 [==============================] - 5s 674us/sample - loss: 0.3581 - accuracy: 0.8521
Epoch 7/150
8000/8000 [==============================] - 5s 674us/sample - loss: 0.3550 - accuracy: 0.8541
Epoch 8/150
8000/8000 [==============================] - 5s 665us/sample - loss: 0.3514 - accuracy: 0.8583
Epoch 9/150
8000/8000 [==============================] - 5s 664us/sample - loss: 0.3494 - accuracy: 0.8597 - loss: 0.3497 

### Defining predict function

In [12]:
def predict_tensorflow(data_path,test_data,model):
    import pickle
    import numpy as np
    from tensorflow import keras
    from tensorflow.keras.models import load_model
    
    #loading the X_test and y_test
    with open(f'{data_path}/{test_data}', 'rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    X_test, y_test = test_data
    #loading the model
    classifier = load_model(f'{data_path}/{model}')

    #Evaluate the model and print the results
    test_loss, test_acc = classifier.evaluate(X_test,  y_test, verbose=0)
    
    #model's prediction on test data
    y_pred = classifier.predict(X_test)
    # create a threshold for the confution matrics
    y_pred=(y_pred>0.5)

    #saving the test_loss and test_acc
    with open(f'{data_path}/performance.txt', 'w') as f:
        f.write("Test_loss: {}, Test_accuracy: {} ".format(test_loss,test_acc))

    #saving the predictions
    with open(f'{data_path}/results.txt', 'w') as result:
        result.write(" Prediction: {}, Actual: {} ".format(y_pred,y_test.astype(np.bool)))

In [13]:
predict_tensorflow(output_dir,"test_data", "model")

### creating the components from the python functions defined above

In [14]:
# create light weight components
obtain_data_op = kfp.components.create_component_from_func(obtain_data,base_image="python:3.7.1")
preprocess_op = kfp.components.create_component_from_func(preprocess,base_image="python:3.7.1")
train_op = kfp.components.create_component_from_func(train_tensorflow, base_image="tensorflow/tensorflow:latest-gpu-py3")
predict_op = kfp.components.create_component_from_func(predict_tensorflow, base_image="tensorflow/tensorflow:latest-gpu-py3")

In [15]:
# create client that would enable communication with the Pipelines API server 
client = kfp.Client()

ERROR:root:Failed to get healthz info attempt 1 of 5.
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.6/site-packages/kfp/_client.py", line 312, in get_kfp_healthz
    response = self._healthz_api.get_healthz()
  File "/home/jovyan/.local/lib/python3.6/site-packages/kfp_server_api/api/healthz_service_api.py", line 77, in get_healthz
    return self.get_healthz_with_http_info(**kwargs)  # noqa: E501
  File "/home/jovyan/.local/lib/python3.6/site-packages/kfp_server_api/api/healthz_service_api.py", line 162, in get_healthz_with_http_info
    collection_formats=collection_formats)
  File "/home/jovyan/.local/lib/python3.6/site-packages/kfp_server_api/api_client.py", line 383, in call_api
    _preload_content, _request_timeout, _host)
  File "/home/jovyan/.local/lib/python3.6/site-packages/kfp_server_api/api_client.py", line 202, in __call_api
    raise e
  File "/home/jovyan/.local/lib/python3.6/site-packages/kfp_server_api/api_client.py", line 199, in __call_ap

TimeoutError: Failed getting healthz endpoint after 5 attempts.

In [ ]:
# define pipeline
@dsl.pipeline(name="Churn Pipeline", description="Performs Preprocessing, training and prediction of churn rate")

# Define parameters to be fed into pipeline
def churn_lightweight_tensorflow_pipeline(data_path: str,
                                          working_data: str,
                                         train_data: str,
                                         test_data:str,
                                         model:str):
    
    # Define volume to share data between components.
    volume_op = dsl.VolumeOp(
    name="data_volume",
    resource_name="data-volume",
    size="1Gi",
    modes=dsl.VOLUME_MODE_RWO)

    #create obtain data component
    obtain_data_container = obtain_data_op(data_path, working_data).add_pvolumes({data_path: volume_op.volume})
    # Create preprocess components.
    preprocess_container = preprocess_op(data_path, working_data, train_data, test_data).add_pvolumes({data_path: obtain_data_container.pvolume})
    # Create train component.
    train_container = train_op(data_path, train_data, model).add_pvolumes({data_path: preprocess_container.pvolume})
    # Create prediction component.
    predict_container = predict_op(data_path, test_data, model).add_pvolumes({data_path: train_container.pvolume})
    
    # Print the result of the prediction
    result_container = dsl.ContainerOp(
        name="print_prediction",
        image='library/bash:4.4.23',
        pvolumes={data_path: predict_container.pvolume},
        arguments=['cat', f'{data_path}/results.txt']
        )

In [ ]:
DATA_PATH = '/mnt'
DATA = "working_data"
TRAIN_DATA = "train_data"
TEST_DATA = "test_data"
MODEL_FILE= "classifier.h5"


pipeline_func = churn_lightweight_tensorflow_pipeline

experiment_name = 'churn_prediction_tensorflow_lightweight'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH,
             "working_data": DATA,
            "train_data": TRAIN_DATA,
            "test_data": TEST_DATA,
            "model":MODEL_FILE}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func,  
  '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)
